# Mechanismy pozornosti a transformery

Jednou z hlavních nevýhod rekurentních sítí je, že všechna slova v sekvenci mají stejný vliv na výsledek. To vede k suboptimálnímu výkonu u standardních modelů LSTM encoder-decoder pro úlohy typu sekvence na sekvenci, jako je rozpoznávání pojmenovaných entit nebo strojový překlad. Ve skutečnosti mají konkrétní slova ve vstupní sekvenci často větší vliv na výstupy než jiná.

Zvažme model sekvence na sekvenci, například strojový překlad. Tento model je implementován pomocí dvou rekurentních sítí, kde jedna síť (**encoder**) zkomprimuje vstupní sekvenci do skrytého stavu a druhá síť (**decoder**) tento skrytý stav rozvine do přeloženého výsledku. Problém tohoto přístupu spočívá v tom, že konečný stav sítě má potíže s uchováním informací z počátku věty, což vede k nízké kvalitě modelu u dlouhých vět.

**Mechanismy pozornosti** poskytují způsob, jak vážit kontextuální vliv jednotlivých vstupních vektorů na každou výstupní predikci RNN. Toho je dosaženo vytvořením zkratek mezi mezistavy vstupní RNN a výstupní RNN. Tímto způsobem, při generování výstupního symbolu $y_t$, bereme v úvahu všechny skryté stavy vstupu $h_i$ s různými váhovými koeficienty $\alpha_{t,i}$. 

![Obrázek zobrazující model encoder/decoder s aditivní vrstvou pozornosti](../../../../../translated_images/cs/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder s mechanismem aditivní pozornosti podle [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citováno z [tohoto blogového příspěvku](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matice pozornosti $\{\alpha_{i,j}\}$ reprezentuje míru, do jaké určitá vstupní slova ovlivňují generování konkrétního slova ve výstupní sekvenci. Níže je příklad takové matice:

![Obrázek zobrazující ukázkové zarovnání nalezené modelem RNNsearch-50, převzato z Bahdanau - arviz.org](../../../../../translated_images/cs/bahdanau-fig3.09ba2d37f202a6af.webp)

*Obrázek převzatý z [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Obr. 3)*

Mechanismy pozornosti jsou zodpovědné za velkou část současného nebo téměř současného stavu techniky v oblasti zpracování přirozeného jazyka. Přidání pozornosti však výrazně zvyšuje počet parametrů modelu, což vedlo k problémům se škálováním u RNN. Klíčovým omezením škálování RNN je, že rekurentní povaha těchto modelů ztěžuje dávkování a paralelizaci tréninku. V RNN musí být každý prvek sekvence zpracován v sekvenčním pořadí, což znamená, že paralelizace není snadná.

Přijetí mechanismů pozornosti v kombinaci s tímto omezením vedlo k vytvoření dnes známých modelů Transformer, které představují současný stav techniky, od BERT po OpenGPT3.

## Modely Transformer

Namísto předávání kontextu každé předchozí predikce do dalšího kroku hodnocení používají **modely Transformer** **poziční kódování** a **pozornost**, aby zachytily kontext daného vstupu v rámci poskytnutého textového okna. Obrázek níže ukazuje, jak poziční kódování s pozorností dokáže zachytit kontext v rámci daného okna.

![Animovaný GIF zobrazující, jak jsou hodnocení prováděna v modelech Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Protože každá vstupní pozice je mapována nezávisle na každou výstupní pozici, transformery umožňují lepší paralelizaci než RNN, což umožňuje vytvářet mnohem větší a expresivnější jazykové modely. Každá hlava pozornosti může být použita k učení různých vztahů mezi slovy, což zlepšuje výsledky úloh zpracování přirozeného jazyka.

## Vytvoření jednoduchého modelu Transformer

Keras neobsahuje vestavěnou vrstvu Transformer, ale můžeme si ji vytvořit sami. Stejně jako dříve se zaměříme na klasifikaci textu z datasetu AG News, ale stojí za zmínku, že modely Transformer dosahují nejlepších výsledků u složitějších úloh zpracování přirozeného jazyka.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nové vrstvy v Keras by měly dědit třídu `Layer` a implementovat metodu `call`. Začněme s vrstvou **Positional Embedding**. Použijeme [nějaký kód z oficiální dokumentace Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Budeme předpokládat, že všechny vstupní sekvence doplníme na délku `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Tato vrstva se skládá ze dvou vrstev `Embedding`: pro vkládání tokenů (způsobem, který jsme již diskutovali) a pozic tokenů. Pozice tokenů jsou vytvořeny jako posloupnost přirozených čísel od 0 do `maxlen` pomocí `tf.range`, a poté jsou předány přes vrstvu embedding. Dva výsledné vektorové embeddingy jsou následně sečteny, čímž vznikne pozičně vložená reprezentace vstupu o tvaru `maxlen`$\times$`embed_dim`.

Nyní implementujeme blok transformátoru. Ten bude přijímat výstup z dříve definované vrstvy embedding:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Nyní jsme připraveni definovat kompletní model transformátoru:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Modely

**BERT** (Bidirectional Encoder Representations from Transformers) je velmi rozsáhlá vícevrstvá transformátorová síť s 12 vrstvami pro *BERT-base* a 24 vrstvami pro *BERT-large*. Model je nejprve předtrénován na velkém korpusu textových dat (WikiPedia + knihy) pomocí nesupervizovaného učení (předpovídání maskovaných slov ve větě). Během předtrénování model získává významnou úroveň porozumění jazyku, kterou lze následně využít s jinými datovými sadami pomocí jemného ladění. Tento proces se nazývá **transferové učení**.

![obrázek z http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/cs/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Existuje mnoho variant architektur Transformerů, včetně BERT, DistilBERT, BigBird, OpenGPT3 a dalších, které lze jemně doladit.

Podívejme se, jak můžeme použít předtrénovaný model BERT k řešení našeho tradičního problému klasifikace sekvencí. Využijeme myšlenku a část kódu z [oficiální dokumentace](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Pro načtení předtrénovaných modelů použijeme **Tensorflow hub**. Nejprve načtěme vektorizační nástroj specifický pro BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Je důležité použít stejný vektorizér, jaký byl použit při trénování původní sítě. BERT vektorizér vrací tři komponenty:
* `input_word_ids`, což je sekvence čísel tokenů pro vstupní větu
* `input_mask`, která ukazuje, která část sekvence obsahuje skutečný vstup a která je vyplněna paddingem. Je podobná masce vytvořené vrstvou `Masking`
* `input_type_ids` se používá pro úlohy jazykového modelování a umožňuje specifikovat dvě vstupní věty v jedné sekvenci.

Poté můžeme vytvořit instanci extraktoru funkcí BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Takže BERT vrstva vrací několik užitečných výsledků:
* `pooled_output` je výsledkem zprůměrování všech tokenů v sekvenci. Můžete si jej představit jako inteligentní sémantické vnoření celé sítě. Je ekvivalentní výstupu vrstvy `GlobalAveragePooling1D` v našem předchozím modelu.
* `sequence_output` je výstup poslední transformátorové vrstvy (odpovídá výstupu `TransformerBlock` v našem modelu výše).
* `encoder_outputs` jsou výstupy všech transformátorových vrstev. Protože jsme nahráli 4-vrstvý BERT model (jak pravděpodobně odhadnete z názvu, který obsahuje `4_H`), má 4 tenzory. Poslední z nich je stejný jako `sequence_output`.

Nyní definujeme end-to-end klasifikační model. Použijeme *funkční definici modelu*, kdy definujeme vstup modelu a poté poskytneme sérii výrazů pro výpočet jeho výstupu. Také nastavíme váhy BERT modelu jako netrénovatelné a budeme trénovat pouze finální klasifikátor:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Přestože je zde jen málo trénovatelných parametrů, proces je poměrně pomalý, protože extraktor příznaků BERT je výpočetně náročný. Vypadá to, že jsme nebyli schopni dosáhnout rozumné přesnosti, buď kvůli nedostatku tréninku, nebo kvůli nedostatku parametrů modelu.

Zkusme odemknout váhy BERT a trénovat je také. To vyžaduje velmi malou rychlost učení a také opatrnější tréninkovou strategii s **warmup**, za použití optimalizátoru **AdamW**. K vytvoření optimalizátoru použijeme balíček `tf-models-official`:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Jak vidíte, trénink probíhá poměrně pomalu - ale možná budete chtít experimentovat a trénovat model několik epoch (5-10) a zjistit, zda dosáhnete nejlepšího výsledku ve srovnání s přístupy, které jsme použili dříve.

## Knihovna Huggingface Transformers

Dalším velmi běžným (a o něco jednodušším) způsobem použití modelů Transformer je [balíček HuggingFace](https://github.com/huggingface/), který poskytuje jednoduché stavební bloky pro různé úlohy NLP. Je dostupný jak pro Tensorflow, tak pro PyTorch, další velmi populární framework pro neuronové sítě.

> **Note**: Pokud vás nezajímá, jak knihovna Transformers funguje - můžete přeskočit na konec tohoto notebooku, protože neuvidíte nic zásadně odlišného od toho, co jsme udělali výše. Budeme opakovat stejné kroky při trénování modelu BERT pomocí jiné knihovny a podstatně většího modelu. Proces tedy zahrnuje poměrně dlouhý trénink, takže možná budete chtít jen projít kód. 

Podívejme se, jak lze náš problém vyřešit pomocí [Huggingface Transformers](http://huggingface.co).


První věc, kterou musíme udělat, je vybrat model, který budeme používat. Kromě několika vestavěných modelů obsahuje Huggingface [online repozitář modelů](https://huggingface.co/models), kde můžete najít mnoho dalších předtrénovaných modelů od komunity. Všechny tyto modely lze načíst a použít jednoduše zadáním názvu modelu. Všechny potřebné binární soubory pro model budou automaticky staženy.

V určitých situacích budete potřebovat načíst vlastní modely, v takovém případě můžete specifikovat adresář, který obsahuje všechny relevantní soubory, včetně parametrů pro tokenizer, soubor `config.json` s parametry modelu, binární váhy atd.

Z názvu modelu můžeme vytvořit jak model, tak tokenizer. Začněme s tokenizerem:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objekt `tokenizer` obsahuje funkci `encode`, kterou lze přímo použít k zakódování textu:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Můžeme také použít tokenizer k zakódování sekvence způsobem vhodným pro předání modelu, tj. včetně polí `token_ids`, `input_mask` atd. Můžeme také specifikovat, že chceme Tensorflow tensory poskytnutím argumentu `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

V našem případě budeme používat předtrénovaný model BERT nazvaný `bert-base-uncased`. *Uncased* znamená, že model nerozlišuje velká a malá písmena.

Při trénování modelu je potřeba poskytnout tokenizovanou sekvenci jako vstup, a proto navrhneme datový zpracovatelský proces. Jelikož `tokenizer.encode` je funkce v Pythonu, použijeme stejný přístup jako v předchozí jednotce, kdy ji voláme pomocí `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nyní můžeme načíst skutečný model pomocí balíčku `BertForSequenceClassification`. To zajišťuje, že náš model již má požadovanou architekturu pro klasifikaci, včetně finálního klasifikátoru. Uvidíte varovnou zprávu, která uvádí, že váhy finálního klasifikátoru nejsou inicializovány a model by vyžadoval předtrénování - to je naprosto v pořádku, protože přesně to se chystáme udělat!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Jak můžete vidět z `summary()`, model obsahuje téměř 110 milionů parametrů! Pravděpodobně, pokud chceme jednoduchý klasifikační úkol na relativně malém datasetu, nechceme trénovat základní vrstvu BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nyní jsme připraveni začít s tréninkem!

> **Poznámka**: Trénování plnohodnotného modelu BERT může být velmi časově náročné! Proto ho budeme trénovat pouze na prvních 32 dávkách. Toto slouží pouze k ukázce, jak nastavit trénink modelu. Pokud máte zájem vyzkoušet plnohodnotné trénování - stačí odstranit parametry `steps_per_epoch` a `validation_steps` a připravit se na čekání!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Pokud zvýšíte počet iterací, počkáte dostatečně dlouho a provedete trénink po několik epoch, můžete očekávat, že klasifikace pomocí BERT nám poskytne nejlepší přesnost! Je to proto, že BERT už velmi dobře rozumí struktuře jazyka, a my potřebujeme pouze doladit finální klasifikátor. Nicméně, protože BERT je velký model, celý proces tréninku trvá dlouho a vyžaduje značnou výpočetní sílu! (GPU, a ideálně více než jedno).

> **Note:** V našem příkladu jsme používali jeden z nejmenších předtrénovaných modelů BERT. Existují větší modely, které pravděpodobně přinesou lepší výsledky.


## Shrnutí

V této jednotce jsme se seznámili s nejnovějšími modelovými architekturami založenými na **transformerech**. Použili jsme je pro úlohu klasifikace textu, ale podobně mohou být BERT modely využity pro extrakci entit, odpovídání na otázky a další úlohy z oblasti NLP.

Modely založené na transformerech představují současný špičkový standard v NLP a ve většině případů by měly být první volbou, se kterou začnete experimentovat při implementaci vlastních řešení NLP. Nicméně pochopení základních principů rekurentních neuronových sítí, o kterých jsme hovořili v tomto modulu, je nesmírně důležité, pokud chcete vytvářet pokročilé neuronové modely.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
